In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import keras
from src.nimbus_analysis import autoencoder


2025-11-29 00:38:22.149320: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-29 00:38:22.178657: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 00:38:22.664489: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
with open('./datasets/train_X.pickle', 'rb') as file:
    train_X = pickle.load(file)
    
train_X = train_X[:, 120:160, 130:170] #take region of interest only
#rescale
train_X = train_X.astype('float32')/np.max(train_X)

#impute zero with median (zero exclusive)
nonzero_median = np.median(train_X[train_X > 0])
train_X[train_X == 0] = nonzero_median

train_y = train_X

#shift by 1 day between input and output
train_y = train_y[1:]
train_X = train_X[:-1]

In [3]:


autoencoder = autoencoder.regional_deep_conv_autoencoder()

W0000 00:00:1764376703.416354   27385 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder.fit(train_X, train_y,
                epochs=10,
                shuffle=False)

Epoch 1/10
 39/144 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0039  

2025-11-29 00:38:24.211939: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0040
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0036
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0035
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0031
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0025
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0016
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0013
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0013
Epoch 9/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0012
Epoch 10/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0011


In [5]:
autoencoder.summary()

Model: "regional_deep_conv_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 5, 5, 8)        │         6,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 40, 40, 1)      │        14,873 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,933 (245.84 KB)

 Trainable params: 20,977 (81.94 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 41,956 (163.89 KB)

In [6]:
autoencoder.save('regional_autoencoder.keras')